In [1]:
from datosipc import datosIPC
import pandas as pd

In [2]:
def export_to_longtable(df: pd.DataFrame, filename: str, header: bool=True, decimals: int=2):
    """
    Exporta un dataframe a un longtable de LaTeX en un archivo.
    
    Parameters:
    df (pandas.DataFrame): DataFrame a exportar.
    filename (str): Nombre del archivo a guardar.
    caption (str): Leyenda de la tabla.
    column_format (str): Cadena que describe el formato de las columnas.
    header (bool): Si True, muestra el encabezado de la tabla.
    decimals (int): Cantidad de decimales para los números. Si el valor no es numérico, se muestra tal cual.
    """
    with open(filename, 'w', encoding="utf-8") as f:
        f.write('\\begin{longtable}{' + 'c'*len(df.columns) + '}\n')
        f.write('\\toprule\n')
        
        # Escribir el encabezado
        if header:
            f.write(' & '.join([col.replace('_', '\\_') for col in df.columns]))
            f.write('\\\\\\midrule\n')
        
        f.write('\\endfirsthead\n')
        f.write('\\multicolumn{' + str(len(df.columns)) + '}{c}{{\\bfseries \\tablename\\ \\thetable{} -- '
                'Continuación de la página anterior}}\\\\\n')
        f.write('\\toprule\n')
        if header:
            f.write(' & '.join([col.replace('_', '\\_') for col in df.columns]))
            f.write('\\\\\\midrule\n')
        f.write('\\endhead\n')
        f.write('\\midrule\n')
        f.write('\\multicolumn{' + str(len(df.columns)) + '}{r}{{Continúa en la siguiente página}}\\\\\n')
        f.write('\\endfoot\n')
        f.write('\\bottomrule\n')
        f.write('\\endlastfoot\n')
        
        # Escribir los datos
        import re
        for _, row in df.iterrows():
            values = []
            for value in row:
                if pd.isna(value):
                    values.append('')
                elif isinstance(value, (int, float)):
                    valor = round(value, decimals)
                    if re.match(r"^[-]0\.0*$", str(valor)):
                        valor = int(valor)
                    valor = '{:.{}f}'.format(valor, decimals)
                    values.append(valor)
                else:
                    values.append(str(value))
            f.write(' & '.join(values))
            f.write('\\\\\n')
        
        f.write('\\end{longtable}\n')

In [3]:
p = datosIPC(2023, 4, dbBackup=True)

In [4]:
df = p.tabla_series_historicas()[0]
df.columns

Index(['Fecha', 'IPC', 'Variación mensual', 'Ritmo inflacionario'], dtype='object')

In [5]:
export_to_longtable(df=df, filename='prueba.tex')

In [4]:
template = """
{{\\Bold \\LARGE EQUIPO RESPONSABLE}}\\\\[2cm]
{team_sections}
"""

team_section = """
{{\\Bold \\large \\color{{color1!89!black}} {team_title}}}}\\\\[0.2cm]
{team_members}\\\\[0.8cm]
"""

def create_file(data):
    team_sections = ''
    for key, value in data.items():
        members = '\\\\\n'.join(value)
        team_sections += team_section.format(team_title=key, team_members=members)
    return template.format(team_sections=team_sections)

data = {
    "EQUIPO TÉCNICO": ("Brenda Izabel Miranda Consuegra", "Werner Santos Salguero García", "Hugo Allan García Monterrosa", "Henry Efraín Coc Ramírez", "Eslie Mercedes Pérez Ruiz", "Donald Estuardo Schaad Moz", "Francisco Javier Romero Celis"),
    "DIAGRAMACIÓN Y DISEÑO": ("Andrea Michelle Rojas Salvatierra", "Luis Alfredo Alvarado Rodríguez"),
}

print(create_file(data))


ValueError: Single '}' encountered in format string